# Calculate CIFAR10 Image Histogram

Before training the image classifier, the PyTorch example follows the following steps: 

* **Prepare Data**
    * Load and normalize the CIFAR10 training and test datasets using torchvision
    
* **Training**
    * Define a Convolutional Neural Network
    * Define a loss function
    * Train the network on the training data
    * Test the network on the test data
    
We will add another step to calculate the data histogram and compare the local (site) histogram and global histograms. So the above steps become


* **Prepare Data**
    * Load and normalize the CIFAR10 training and test datasets using torchvision

* **Data Statistics**
    * Calculate data stastics: image intensity histograms
    
* **Training**
    * Define a Convolutional Neural Network
    * Define a loss function
    * Train the network on the training data
    * Test the network on the test data



## Setup NVFLARE

Follow [Getting Started](../../../../getting_started/readme.ipynb) to set up a virtual environment and install NVFLARE.


## Install requirements
assuming the current directory is 'cifar10/stats'

In [ ]:
!pwd

In [ ]:
%pip install -r requirements.txt

## Prepare Data

Generally, when you have to deal with image, text, audio or video data, you can use standard python packages that load data into a numpy array. Then you can convert this array into a torch.*Tensor. Torch provided a package called torchvision, that has data loaders for common datasets such as ImageNet, CIFAR10, MNIST, etc. and data transformers for images, viz., torchvision.datasets and torch.utils.data.DataLoader.

For CIFAR10 dataset, it has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. 
The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

![cifar10](../data/cifar10.png)


The output of torchvision datasets are PILImage images of range [0, 1]. We transform them to Tensors of normalized range [-1, 1].





In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

CIFAR10_ROOT = "/tmp/nvflare/data/cifar10"

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 6

trainset = torchvision.datasets.CIFAR10(root=CIFAR10_ROOT, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=CIFAR10_ROOT, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Once you have extract the data from zip file, you can check the directory

In [ ]:
ls -al {CIFAR10_ROOT}

Let's explore the data first

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)

images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

# dimension and shapes
 
# Display image and label.
print(f"\nFeature batch shape: {images.size()}")
print(f"Labels batch shape: {labels.size()} \n")


print("train datasize =", len(trainset))
print("test datasize =", len(testset))
    

We can see the images has shape of [batch, channel, rows, cols] = [6,3,32,32]

## Download data in script
We have prepared python script to download the data as well. 
```
python ../data/download.py  --dataset_path <data_path>
```
if dataset_path is not specified, it default to CIFAR10_ROOT

In [ ]:
! python ../data/download.py


## Create Local Image Intensity Histogram Calculator

We ignored all other statistical calculations (mean, standard deviation, etc.) as they do not apply. all methods have default implementations.


In [ ]:
from typing import Dict, List

import numpy as np
import torchvision

from nvflare.apis.fl_context import FLContext
from nvflare.app_common.abstract.statistics_spec import Bin, DataType, Feature, Histogram, HistogramType, Statistics
 
# the dataset path    
CIFAR10_ROOT = "/tmp/nvflare/data/cifar10"


class ImageStatistics(Statistics):

    def __init__(self, 
                 data_root: str = CIFAR10_ROOT, 
                 batch_size: int = 4):
        """local image intensity calculator.

        Args:
            dataset_path: directory with local image data.
         Returns:
            Histogram of local statistics`
        """
        super().__init__()
        self.dataset_path = data_root
        self.batch_size = batch_size
        
        # there are three color channels : RGB, each corresponding to each channel index
        # we are going treat each channel as one feature, the feature Ids are corresponding to tensor channel index. 
        # The feature name is named "red", "gree", "blue" (RGB). 
        
        self.features_ids = { "red": 0, "green": 1,"blue": 2}
        self.image_features  = [Feature("red", DataType.FLOAT),
                                Feature("green", DataType.FLOAT),
                                Feature("blue", DataType.FLOAT)]
        self.dataset_lengths = {}
        self.loaders = {}

        self.client_name = None
        self.fl_ctx = None



    def initialize(self, fl_ctx: FLContext):

        # FLContext is context information for the client side NVFLARE engine. 
        # it includes many runtime information. 
        # Here we only interested in client site name. 
        # fl_ctx.get_identity_name() will return the client's name
        
        self.fl_ctx = fl_ctx
        self.client_name = "local_client" if fl_ctx is None else fl_ctx.get_identity_name()
        
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        trainset = torchvision.datasets.CIFAR10(root=self.dataset_path, train=True, download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root=self.dataset_path, train=False,  download=True, transform=transform)
        self.dataset_lengths = {"train": len(trainset), "test":len(testset)}
        
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=self.batch_size, shuffle=True, num_workers=2)
        testloader = torch.utils.data.DataLoader(testset, batch_size=self.batch_size, shuffle=False, num_workers=2)
        self.loaders = {"train": trainloader, "test": testloader}

    def features(self) -> Dict[str, List[Feature]]:
        return {"train": self.image_features, 
                "test":  self.image_features}

    def count(self, dataset_name: str, feature_name: str) -> int:
        return self.dataset_lengths[dataset_name]
        
 
    def histogram(self,
                  dataset_name: str,
                  feature_name: str, 
                  num_of_bins: int, 
                  global_min_value: float, 
                  global_max_value: float) -> Histogram:
     
        print(f"calculating image intensity histogram for client {self.client_name}")
        channel = self.features_ids[feature_name]
        
        # get the inputs; data is a list of [inputs, labels]
        histogram_bins: List[Bin] = []
        bin_edges = []
        histogram = np.zeros(num_of_bins, dtype=float)

        for inputs, _ in self.loaders[dataset_name]:
            for img in inputs:
                counts, bin_edges = np.histogram(img[channel, : , :],
                                                 bins=num_of_bins,
                                                 range=(global_min_value, global_max_value))
                histogram += counts

        for i in range(num_of_bins):
            low_value = bin_edges[i]
            high_value = bin_edges[i + 1]
            bin_sample_count = histogram[i]
            histogram_bins.append(Bin(low_value=low_value, high_value=high_value, sample_count=bin_sample_count))

        return Histogram(HistogramType.STANDARD, histogram_bins)




Let's test if the code works. 

In [ ]:
hist_cal = ImageStatistics()

hist_cal.initialize(fl_ctx = None)
features = hist_cal.features()

In [ ]:
hist_cal.count("train", "red")

In [ ]:
hist_cal.histogram("train", "red", 20, 0, 256)

The code is working. Let's set up an NVFLARE job in federated computing.

## Create Federated Histogram Job
We are going to use NVFLARE job API to construct a FedJob

In [ ]:
! cat image_stats_job.py


## Run Job in FL Simulator


**Run job.simulator_run()**


In [ ]:
! python image_stats_job.py -w /tmp/nvflare/image_stats -n 2 -d /tmp/nvflare/data/cifar10 -o "statistics/image_statistics.json"



**Run Job using Simulator CLI**

```
! python image_stats_job.py -co -j /tmp/nvflare/jobs/stats_image -n 2
! nvflare simulator /tmp/nvflare/jobs/stats_image -w /tmp/nvflare/image_stats -n 2 -t 2

```



**Examine the result**

Note that the result is written at 

**/tmp/nvflare/image_stats/server/simulate_job/statistics/image_statistics.json**

In [ ]:
!ls -al /tmp/nvflare/image_stats/server/simulate_job/statistics/image_statistics.json


## Visualization
We can visualize the results easly via the visualizaiton notebook. Before we do that, we need to copy the data to the notebook directory 


In [ ]:
! cp /tmp/nvflare/image_stats/server/simulate_job/statistics/image_statistics.json ./.

In [ ]:

import json
import pandas as pd
from nvflare.app_opt.statistics.visualization.statistics_visualization import Visualization
with open('image_statistics.json', 'r') as f:
    data = json.load(f)

vis = Visualization()
vis.show_stats(data = data)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100%  depth:100% !important; }</style>"))

In [ ]:
vis.show_histograms(data = data, plot_type="main")

The global and local histograms show no differences because we are using the same dataset for all clients.

## We are done !
Congratulations! you have just completed the federated stats image histogram calulation. 

If you would like to see another example of federated statistics calculations and configurations, please checkout [federated_statistics](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/federated-statistics) and [fed_stats with spleen_ct_segmentation](https://github.com/NVIDIA/NVFlare/tree/main/integration/monai/examples/spleen_ct_segmentation_sim)

Let's move on to the next examples and see how can we train the image classifier using pytorch with CIFAR10 data.
First we will look at the [sag](../sag/sag.ipynb) example, which illustrates how to use the ScatterAndGather workflow for FedAvg with the Client API.
